In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input, concatenate, Dropout, Reshape, Conv1D, MaxPooling1D, Flatten, Activation
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras.losses import mean_absolute_percentage_error
from keras.callbacks import ModelCheckpoint, EarlyStopping

import sys
from ZernikeCNN import ZernikeConv, resampleGraph_expand, Normalized_resampleGraph_expand, ZernikeDecomp
import numpy as np
import os
import scipy.io as sio
import scipy.stats


Using TensorFlow backend.


In [2]:
import math

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def random_rotation(x_Feature):
    x_Feature = x_Feature.transpose()
    
    axis_y = [0, 1, 0]
    theta_list = [-np.pi+np.pi/16*k for k in range(32)]
    theta_y = theta_list[np.random.randint(len(theta_list))]
    Rot = rotation_matrix(axis_y, theta_y)
    
    x_Feature = np.matmul(Rot,x_Feature)
    x_Feature = x_Feature.transpose()
    
    return x_Feature

In [3]:
import scipy.io as sio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], sio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, sio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict


In [4]:
def load_input_feature(path):
    sampled_infor = loadmat(path)
    input_feature = sampled_infor['sampled_surface']['X'].astype('float32')
    return input_feature

def load_patchfeature_input(path):
    ZerNet_preproc = loadmat(path)
    Input_Patch_Features = ZerNet_preproc['ZerNet_preproc']['scale_1']['ZerPatch']['FeaVec'].astype('float32')
    return Input_Patch_Features

def load_mapping_infor(path):
    sampled_infor = loadmat(path)
    mesh_faces = (sampled_infor['sampled_surface']['F']-1).astype('int32')
    resample_graph_I = (sampled_infor['sampled_surface']['I']-1).astype('int32')
    resample_graph_B = sampled_infor['sampled_surface']['B'].astype('float32')
    return mesh_faces, resample_graph_I, resample_graph_B

def load_output_label(path):
    sampled_infor = loadmat(path)
    label_out = (sampled_infor['sampled_surface']['label_X']-1).astype('int32')
    return label_out

def load_Zerpatch_input(path, scalename):
    ZerNet_preproc = loadmat(path)
    Zerbases = ZerNet_preproc['ZerNet_preproc'][scalename]['ZerPatch']['bases'].astype('float32')
    resampleGraph = (ZerNet_preproc['ZerNet_preproc'][scalename]['ZerPatch']['resamplePids']-1).astype('int32')
    scale ={'Zerbases':Zerbases,
            'resampleGraph':resampleGraph}
    return scale

In [5]:
def create_data(patch_path, sampled_path, argument=False):
    scale_1 = load_Zerpatch_input(patch_path, 'scale_1')
    Zerbases = scale_1['Zerbases']
    resampleGraph = scale_1['resampleGraph']
    Disk_Feature = load_patchfeature_input(patch_path)  
    label_out = load_output_label(sampled_path)
    return Disk_Feature,resampleGraph,Zerbases,label_out

# def create_data(patch_path, sampled_path, argument):
#     scale_1 = load_Zerpatch_input(patch_path, 'scale_1')
#     Zerbases = scale_1['Zerbases']
#     resampleGraph = scale_1['resampleGraph']
#     input_feature = load_input_feature(sampled_path)
#     if argument:
#         #random scaleing and rotation
# #         scaling_factor = (1.15-0.85)*random.uniform(0, 1)+0.85
# #         input_feature = scaling_factor*input_feature
#           input_feature = random_rotation(input_feature)  
#     label_out = load_output_label(sampled_path)
#     return input_feature,resampleGraph,Zerbases,label_out

In [6]:
import keras
import numpy as np
import os
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_model_names, patches_folder, sampled_folder, argument=False, batch_size=1, basis_num = 21, 
                 num_input_channels=3, shuffle=True):
        'Initialization'
        self.list_model_names = list_model_names
        self.patches_folder = patches_folder
        self.sampled_folder = sampled_folder
        self.argument = argument
        self.basis_num = basis_num
        self.batch_size = batch_size
        self.num_input_channels = num_input_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_model_names) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        indexes = self.indexes[index:index+1]

        # Find list of model_names
        list_model_names_temp = [self.list_model_names[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_model_names_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_model_names))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)        

    def __data_generation(self, list_model_names_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        model_name = list_model_names_temp[0]

        # Generate data
        patch_path = os.path.join(self.patches_folder,model_name)
        sampled_path = os.path.join(self.sampled_folder,model_name)
            
        Disk_Feature,resampleGraph,Zerbases,label_out = create_data(patch_path,sampled_path,self.argument)
        mesh_faces, resample_graph_I, resample_graph_B = load_mapping_infor(sampled_path)
        
        x_Disk_Feature = np.expand_dims(Disk_Feature,axis=0)
        x_rG = np.expand_dims(resampleGraph,axis=0)
        x_Zb = np.expand_dims(Zerbases,axis=0)
        
        x_F = np.expand_dims(mesh_faces,axis=0)
        x_I = np.expand_dims(resample_graph_I,axis=0)
        x_B = np.expand_dims(resample_graph_B,axis=0)
        
        X = [x_Disk_Feature, x_rG, x_F, x_I, x_B, x_Zb]
        Y = np.expand_dims(label_out, axis=-1)
        Y = np.expand_dims(Y, axis=0)
        
        return X, Y

In [7]:
# Datasets
patches_folder = './Data/Faust Segmentation Dataset/ZerNet Input/Input ZerPatches' 
sampled_folder = './Data/Faust Segmentation Dataset/UniformSampling_surfaces'

# model_names = os.listdir(patches_folder)
# print(model_names)

model_names = []
for i in range(100):
    model_id = 1000+i;
    model_id = str(model_id)[1:]
    model_name = 'faust_tr_reg_' + model_id + '.mat'
    model_names.append(model_name)
# print(model_names)

val_start_id = 80
val_range = np.array(range(val_start_id,val_start_id+20))

total_range = np.array(range(len(model_names)))
train_range = np.setdiff1d(total_range,val_range)

train_model_names = model_names[train_range[0]:train_range[-1]+1]
val_model_names = model_names[val_range[0]:val_range[-1]+1]

In [8]:
# Parameters
params = {'batch_size': 1,
          'basis_num': 21,
          'num_input_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(train_model_names, patches_folder, sampled_folder, argument=False, **params)
validation_generator = DataGenerator(val_model_names, patches_folder, sampled_folder, argument=False, **params)

train_steps = len(train_model_names)
val_steps = len(val_model_names)

In [9]:
def Initial_ZerConv_block(nf, nrays_expand, x_Disk_Feature, x_Zb, dropout_ratio=0.25):
    Zercoeff = ZernikeDecomp(angular_axis=False)([x_Disk_Feature,x_Zb])
    Features = ZernikeConv(filters = nf, numofrays = nrays_expand, angular_axis = False, 
                           angular_expand=True,angular_pooling=False, activation='relu')(Zercoeff)
    Features = Dropout(dropout_ratio)(Features)
    return Features


def ZerConv_block(nf,nrays_in,x_Feature, x_rG, x_F, x_I, x_B, x_Zb, pooling=False, dropout_ratio=0.25):
    Disk_Features = resampleGraph_expand(angular_axis=True)([x_Feature, x_rG, x_F, x_I, x_B])
    Zercoeff = ZernikeDecomp(angular_axis=True, numofrays=nrays_in)([Disk_Features,x_Zb])
    Features = ZernikeConv(filters = nf, numofrays = nrays_in, angular_axis = True, 
                           angular_expand=False,angular_pooling=pooling, activation='relu')(Zercoeff)  
    Features = Dropout(dropout_ratio)(Features)
    return Features

In [10]:
num_input_channels = 3
cut_num = 50
basis_num = 21
n_classes = 8
nrays = 1

Disk_Features_in = Input(shape=(None,cut_num,num_input_channels,), dtype = 'float32')
resampleGraph = Input(shape=(None,cut_num,), dtype = 'int32', name = 'rG')
F = Input(shape=(None,3), dtype = 'int32', name = 'F')
I = Input(shape=(None,), dtype = 'int32', name = 'I')
B = Input(shape=(None,3,), dtype = 'float32', name = 'B')
Zerbases = Input(shape=(None,cut_num,basis_num,), dtype = 'float32', name = 'Zb')

Features = Initial_ZerConv_block(32,nrays,Disk_Features_in, Zerbases)
Features = ZerConv_block(64, nrays, Features, resampleGraph, F, I, B, Zerbases, pooling=False)
Features = ZerConv_block(128, nrays, Features, resampleGraph, F, I, B, Zerbases, pooling=True)

Features = Conv1D(filters = 256, kernel_size=1, activation='relu')(Features)
Features = Dropout(0.25)(Features)
Features = Conv1D(filters = n_classes, kernel_size=1, activation='softmax')(Features)
model = Model(inputs = [Disk_Features_in,resampleGraph, F, I, B, Zerbases], outputs = Features)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 50, 3)   0                                            
____________________________________________________________________________________________________
Zb (InputLayer)                  (None, None, 50, 21)  0                                            
____________________________________________________________________________________________________
zernike_decomp_1 (ZernikeDecomp) (None, None, 21, 3)   0           input_1[0][0]                    
                                                                   Zb[0][0]                         
____________________________________________________________________________________________________
zernike_conv_1 (ZernikeConv)     (None, None, 1, 32)   2048        zernike_decomp_1[0][0]  

In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss= 'sparse_categorical_crossentropy', optimizer = adam, metrics=['sparse_categorical_accuracy'])

weights_dir = './Trained Models/seg_train_checkpoints'

trained_model_name = 'ZerNet_faust_segmentation'
checkpointer = ModelCheckpoint(filepath = os.path.join(weights_dir, trained_model_name),
                               monitor = 'val_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

earlystopping = EarlyStopping(monitor='val_sparse_categorical_accuracy', min_delta=0.001, patience=35, verbose=0, mode='max')

network_history = model.fit_generator(generator=training_generator, steps_per_epoch=train_steps,
                                      epochs=200, verbose=1,
                                      callbacks=[checkpointer, earlystopping],
                                      validation_data=validation_generator,
                                      validation_steps=val_steps)

In [ ]:
def load_test_data(patches_folder, sampled_folder, model_name):
    patch_path = os.path.join(patches_folder,model_name)
    sampled_path = os.path.join(sampled_folder,model_name)
            
    input_Feature,resampleGraph,Zerbases,label_out = create_data(patch_path,sampled_path,False)
    mesh_faces, resample_graph_I, resample_graph_B = load_mapping_infor(sampled_path)
        
    x_Feature = np.expand_dims(input_Feature,axis=0)
    x_rG = np.expand_dims(resampleGraph,axis=0)
    x_Zb = np.expand_dims(Zerbases,axis=0)
        
    x_F = np.expand_dims(mesh_faces,axis=0)
    x_I = np.expand_dims(resample_graph_I,axis=0)
    x_B = np.expand_dims(resample_graph_B,axis=0)
        
    X = [x_Feature, x_rG, x_F, x_I, x_B, x_Zb]
    Y = np.expand_dims(label_out, axis=-1)
    Y = np.expand_dims(Y, axis=0)
    return X,Y

In [ ]:
predict_folder = './Data/Faust Segmentation Dataset/ZerNet Predict'
model.load_weights(os.path.join(weights_dir,trained_model_name))

for model_name in val_model_names:
    
    test_X, test_Y = load_test_data(patches_folder, sampled_folder, model_name)
    Yp = model.predict(test_X, batch_size=1)
    Yp = np.argmax(Yp, axis=2)
    Yp = np.squeeze(Yp)
    
    sio.savemat(os.path.join(predict_folder, model_name), {'predict_label':Yp})